論文  
https://arxiv.org/abs/2109.04425  
GitHub  
https://github.com/yumingj/Talk-to-Edit  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Talk-to-Edit-demo.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境セットアップ

In [ ]:
# ライブラリダウンロード
!pip install ninja facenet-pytorch scikit-image==0.15.*
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html 

# GitHubからコードを取得
!git clone https://github.com/yumingj/Talk-to-Edit.git

# 学習済みモデルのダウンロード
%cd /content/Talk-to-Edit/download
!wget -c https://www.dropbox.com/s/wlq8plp2qg3sw7i/pretrained_models.zip?dl=0 -O pretrained_models.zip
!unzip pretrained_models.zip
!rm -rf pretrained_models.zip

## 学習済みモデルのダウンロード

In [ ]:
# 学習済みモデルのダウンロード 
%cd /content/Talk-to-Edit/download/pretrained_models/
!wget -c http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
!pip show scikit-image

In [ ]:
## 画像アップロード

In [ ]:
%cd /content/Talk-to-Edit/download/

# 画像アップロード
from google.colab import files
uploaded = files.upload()
uploaded = list(uploaded.keys())
print(uploaded)
file_name = uploaded[0]

In [ ]:
# アップロードした画像をConfigに反映
!sed -i /content/Talk-to-Edit/configs/editing/editing_wo_dialog.yml \
-e "s/is_real_image: False/is_real_image: True/g"

!sed -i /content/Talk-to-Edit/configs/editing/editing_wo_dialog.yml \
-e "s/img_path: .\/download\/.*/img_path: .\/download\/$file_name/g"

## モデルのロード

In [ ]:
%cd /content/Talk-to-Edit

# Config設定
import argparse
import json
import logging
import os.path
import random

import numpy as np
import torch
import torchvision

from models import create_model
from models.utils import save_image
from utils.editing_utils import edit_target_attribute
from utils.logger import get_root_logger
from utils.options import (dict2str, dict_to_nonedict, parse,
                           parse_opt_wrt_resolution)
from utils.util import make_exp_dirs
from utils.inversion_utils import inversion


from PIL import Image

opt = './configs/editing/editing_wo_dialog.yml'
args = argparse.ArgumentParser(description='')
opt = parse(opt, is_train=False)
opt['img_res'] = 1024
opt = parse_opt_wrt_resolution(opt)
try:
  make_exp_dirs(opt)
except Exception as e:
  print(e)

# set up logger
save_log_path = f'{opt["path"]["log"]}'
editing_logger = get_root_logger(
    logger_name='editing',
    log_level=logging.INFO,
    log_file=f'{save_log_path}/editing.log')
editing_logger.info(dict2str(opt))

save_image_path = f'{opt["path"]["visualization"]}'
os.makedirs(save_image_path, exist_ok=True)
  
# ---------- create model ----------
opt['predictor_ckpt'] = './download/pretrained_models/predictor_1024.pth.tar'
field_model = create_model(opt)

## 画像のセットアップ(latent data生成)

In [ ]:
latent_type='upload_image' #@param ['upload_image', 'random'] {allow-input: true}

if latent_type == 'random':
  latent_code = torch.randn(1, 512, device=torch.device('cuda'))
  with torch.no_grad():
    latent_code = field_model.stylegan_gen.get_latent(latent_code)
  latent_code = latent_code.cpu().numpy()

else:
  file_path = "/content/Talk-to-Edit/download/" + file_name
  latent_code = inversion(opt, field_model)

np.save(f'{opt["path"]["visualization"]}/latent_code.npz.npy', latent_code)

from models.utils import save_image
with torch.no_grad():
    start_image, start_label, start_score = \
            field_model.synthesize_and_predict(torch.from_numpy(latent_code).to(torch.device('cuda'))) # noqa

    save_image(start_image, f'{opt["path"]["visualization"]}/start_image.png')

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(
    mpimg.imread(f'{opt["path"]["visualization"]}/start_image.png'))
plt.axis('off')
plt.show()

## 属性変更
Bangs:前髪        0～5(毛量少～多い)  
Eyeglasses:眼鏡   0～5(サイズ小～大)  
No_Beard:ひげ     0～5(毛量少～多い)  
Smiling:笑顔      0～5(笑顔小～大)  
Young:年齢        0～5(年齢若～老)

In [ ]:
# ---------- decide which attribute to edit ----------
target_attribute='No_Beard' #@param ['Bangs', 'Eyeglasses', 'No_Beard', 'Smiling', 'Young'] {allow-input: false}
target_attr_value="5" #@param [0, 1, 2, 3, 4, 5] {allow-input: false}

# initialize attribtue_dict
attribute_dict = {
    "Bangs": start_label[0],
    "Eyeglasses": start_label[1],
    "No_Beard": start_label[2],
    "Smiling": start_label[3],
    "Young": start_label[4],
}

edit_label = {
    'attribute': target_attribute, 
    'target_score': int(target_attr_value)
    }

print_intermediate_result = False
round_idx = 0

edited_latent_code = None

## 属性変更結果出力

In [ ]:
attribute_dict, exception_mode, _, edited_latent_code = edit_target_attribute(
    opt, attribute_dict, edit_label, round_idx, latent_code,
    edited_latent_code, field_model, None,
    print_intermediate_result, display_img=True
    )

if exception_mode != 'normal':
    if exception_mode == 'already_at_target_class':
        editing_logger.info("This attribute is already at the degree that you want. Let's try a different attribute degree or another attribute.")
    elif exception_mode == 'max_edit_num_reached':
        editing_logger.info("Sorry, we are unable to edit this attribute. Perhaps we can try something else.")